# Detecting Anomalous Trades in U.S. Congressional Stock Transactions

**Master's Thesis - Economics, Universidad de San Andrés**

---

## Overview

This notebook implements unsupervised anomaly detection on congressional stock trades. The goal is to identify transactions that deviate significantly from typical trading patterns, which may warrant further investigation for potential informed trading.

**Methodology:**
- Isolation Forest (Liu et al., 2008): detects global outliers
- Local Outlier Factor (Breunig et al., 2000): detects contextual outliers

**Validation approach:**
Since we lack ground-truth labels, we validate by testing whether flagged anomalies exhibit higher cumulative abnormal returns (CAR), consistent with informed trading.

---

## 0. Setup

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.lines import Line2D
import seaborn as sns

# Stats
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage

# ML / Preprocessing
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA

# Anomaly Detection
from pyod.models.iforest import IForest
from pyod.models.lof import LOF

# ============================================================
# PLOT STYLE CONFIGURATION
# Goal: paper-quality figures, clean aesthetic
# ============================================================

plt.rcParams.update({
    'figure.facecolor': 'white',
    'axes.facecolor': 'white',
    'axes.grid': True,
    'grid.alpha': 0.3,
    'grid.linestyle': '--',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.family': 'serif',
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.dpi': 120,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight'
})

# Color palette - muted, professional
COLORS = {
    'primary': '#2C3E50',
    'secondary': '#7F8C8D', 
    'accent': '#E74C3C',
    'success': '#27AE60',
    'warning': '#F39C12',
    'dem': '#3498DB',
    'rep': '#E74C3C',
    'ind': '#95A5A6'
}

print('Setup complete.')

---
## 1. Data Loading and Cleaning

The raw data comes from Stata and has several variables stored as strings that should be numeric. We handle this systematically.

In [ ]:
# Load data
df_raw = pd.read_parquet('congress_merged.parquet')
print(f'Loaded {len(df_raw):,} observations, {df_raw.shape[1]} variables')

In [ ]:
# Working copy
df = df_raw.copy()

# ============================================================
# TYPE CONVERSION
# Many numeric variables were exported as strings from Stata.
# We convert them, coercing errors to NaN.
# ============================================================

numeric_vars = [
    # Returns
    'excess_return', 'return_t', 'abs_return_t', 'return_overnight', 'return_intraday',
    # Momentum
    'momentum_5d', 'momentum_20d', 'momentum_60d', 'momentum_252d',
    # Volatility
    'realized_vol_30d', 'parkinson_vol_30d', 'realized_vol_60d', 
    'vol_of_vol_60d', 'realized_vol_252d',
    # Volume
    'volume_t', 'dollar_volume_t', 'volume_ratio_30d', 'abnormal_volume_30d',
    # Liquidity
    'amihud_illiq_20d', 'roll_spread_30d', 'hl_spread_20d', 'zero_volume_days_30d',
    # Risk
    'beta_252d', 'r2_market_252d',
    # Valuation
    'market_cap', 'price', 'book_value', 'price_to_book', 'ev_to_ebitda',
    # CAR
    'car_raw_30d', 'car_capm_30d', 'car_raw_60d', 'car_capm_60d', 
    'car_raw_90d', 'car_capm_90d'
]

for var in numeric_vars:
    if var in df.columns:
        df[var] = pd.to_numeric(df[var], errors='coerce')

# Date conversion
if 'traded' in df.columns:
    df['traded_date'] = pd.to_datetime(df['traded'], dayfirst=True, errors='coerce')
    df['trade_year'] = df['traded_date'].dt.year
    df['trade_month'] = df['traded_date'].dt.month

if 'filed' in df.columns:
    df['filed_date'] = pd.to_datetime(df['filed'], dayfirst=True, errors='coerce')

# Filing delay: days between trade and disclosure
# Longer delays may indicate strategic timing of disclosure
if 'traded_date' in df.columns and 'filed_date' in df.columns:
    df['filing_delay'] = (df['filed_date'] - df['traded_date']).dt.days
    # Cap extreme values (some errors in data)
    df.loc[df['filing_delay'] < 0, 'filing_delay'] = np.nan
    df.loc[df['filing_delay'] > 365, 'filing_delay'] = np.nan

print(f'Type conversion complete.')

In [ ]:
# Quick check on numeric conversion success
print('Sample of converted numeric variables:')
check_vars = ['car_raw_30d', 'beta_252d', 'realized_vol_30d', 'filing_delay']
for v in check_vars:
    if v in df.columns:
        valid = df[v].notna().sum()
        pct = valid / len(df) * 100
        print(f'  {v}: {valid:,} valid ({pct:.1f}%)')

---
## 2. Exploratory Analysis

In [ ]:
# Basic dimensions
print(f'Total trades: {len(df):,}')
print(f'Unique politicians: {df["name"].nunique():,}')
print(f'Unique tickers: {df["ticker"].nunique():,}')
print(f'Date range: {df["trade_year"].min():.0f} - {df["trade_year"].max():.0f}')

In [ ]:
# ============================================================
# FIGURE 1: Trading Activity Over Time
# ============================================================

fig, ax = plt.subplots(figsize=(10, 4))

yearly = df.groupby('trade_year').size()
ax.bar(yearly.index, yearly.values, color=COLORS['primary'], edgecolor='white', linewidth=0.5)

# Mark STOCK Act (2012)
ax.axvline(2012, color=COLORS['accent'], linestyle='--', linewidth=1.5, alpha=0.8)
ax.text(2012.2, ax.get_ylim()[1]*0.9, 'STOCK Act', fontsize=9, color=COLORS['accent'])

ax.set_xlabel('Year')
ax.set_ylabel('Number of Trades')
ax.set_title('Congressional Stock Trading Activity by Year')

plt.tight_layout()
plt.savefig('fig1_trading_activity.png', dpi=300)
plt.show()

In [ ]:
# ============================================================
# FIGURE 2: Distribution by Party and Chamber
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Party distribution
# Note: party is likely coded as 1=Dem, 2=Rep in Stata
party_map = {1: 'Democrat', 2: 'Republican', 3: 'Independent'}
df['party_label'] = df['party'].map(party_map).fillna('Unknown')

party_counts = df['party_label'].value_counts()
party_colors = [COLORS['dem'] if 'Dem' in p else COLORS['rep'] if 'Rep' in p else COLORS['ind'] 
                for p in party_counts.index]

axes[0].barh(party_counts.index, party_counts.values, color=party_colors, edgecolor='white')
axes[0].set_xlabel('Number of Trades')
axes[0].set_title('Trades by Party')

# Chamber distribution
chamber_map = {1: 'House', 2: 'Senate'}
df['chamber_label'] = df['chamber'].map(chamber_map).fillna('Unknown')

chamber_counts = df['chamber_label'].value_counts()
axes[1].barh(chamber_counts.index, chamber_counts.values, color=COLORS['primary'], edgecolor='white')
axes[1].set_xlabel('Number of Trades')
axes[1].set_title('Trades by Chamber')

plt.tight_layout()
plt.savefig('fig2_party_chamber.png', dpi=300)
plt.show()

In [ ]:
# Transaction type
print('Transaction types:')
print(df['transaction'].value_counts())

### 2.1 Variable Distributions

In [ ]:
# Descriptive statistics for key variables
key_vars = ['car_raw_30d', 'excess_return', 'abnormal_volume_30d', 
            'realized_vol_30d', 'momentum_20d', 'beta_252d', 'filing_delay']
key_vars = [v for v in key_vars if v in df.columns]

df[key_vars].describe().T.round(4)

In [ ]:
# ============================================================
# FIGURE 3: Distribution of Key Variables
# Using kernel density + histogram for better visualization
# ============================================================

fig, axes = plt.subplots(2, 3, figsize=(12, 7))
axes = axes.flatten()

plot_vars = ['car_raw_30d', 'abnormal_volume_30d', 'realized_vol_30d', 
             'momentum_20d', 'beta_252d', 'filing_delay']
plot_vars = [v for v in plot_vars if v in df.columns]

for i, var in enumerate(plot_vars):
    ax = axes[i]
    data = df[var].dropna()
    
    # Winsorize at 1st/99th percentile for visualization
    q01, q99 = data.quantile([0.01, 0.99])
    data_plot = data[(data >= q01) & (data <= q99)]
    
    ax.hist(data_plot, bins=50, density=True, alpha=0.6, color=COLORS['primary'], edgecolor='white')
    
    # Add KDE
    try:
        data_plot.plot.kde(ax=ax, color=COLORS['accent'], linewidth=2)
    except:
        pass
    
    ax.axvline(data.median(), color=COLORS['warning'], linestyle='--', linewidth=1.5, label='Median')
    ax.set_title(var.replace('_', ' ').title())
    ax.set_ylabel('')

# Remove empty subplots if any
for j in range(len(plot_vars), len(axes)):
    axes[j].axis('off')

plt.suptitle('Distribution of Key Features (Winsorized 1%-99%)', fontsize=13, y=1.02)
plt.tight_layout()
plt.savefig('fig3_distributions.png', dpi=300)
plt.show()

### 2.2 Correlation Analysis

In [ ]:
# ============================================================
# FIGURE 4: Correlation Matrix
# We expect high correlation within variable families 
# (e.g., different CAR windows, different volatility measures)
# ============================================================

# Select numeric columns with sufficient non-null values
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Remove identifiers and year variables
exclude = ['trade_year', 'trade_month', 'congress', 'committee_id', 'committee_from_date']
num_cols = [c for c in num_cols if c not in exclude and df[c].notna().sum() > len(df)*0.3]

corr = df[num_cols].corr()

# Create mask for upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

fig, ax = plt.subplots(figsize=(14, 11))
sns.heatmap(corr, mask=mask, cmap='RdBu_r', center=0, 
            square=True, linewidths=0.5, 
            cbar_kws={'shrink': 0.6, 'label': 'Correlation'},
            vmin=-1, vmax=1, ax=ax)
ax.set_title('Correlation Matrix of Numeric Variables', fontsize=13, pad=20)

plt.tight_layout()
plt.savefig('fig4_correlation.png', dpi=300)
plt.show()

In [ ]:
# Identify highly correlated pairs (|r| > 0.8)
# This informs feature selection: we should not include redundant variables

high_corr = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        r = corr.iloc[i, j]
        if abs(r) > 0.8:
            high_corr.append({
                'var1': corr.columns[i],
                'var2': corr.columns[j],
                'correlation': round(r, 3)
            })

if high_corr:
    print('Highly correlated pairs (|r| > 0.8):')
    print(pd.DataFrame(high_corr).sort_values('correlation', ascending=False).to_string(index=False))
else:
    print('No pairs with |r| > 0.8 found.')

---
## 3. Feature Engineering and Selection

For the anomaly detection model, we select features that:
1. Capture different aspects of trading behavior (returns, volume, timing)
2. Avoid redundancy (one variable per highly-correlated family)
3. Have theoretical relevance for detecting informed trading

In [ ]:
# ============================================================
# FEATURE SELECTION RATIONALE
# ============================================================
#
# We select one representative variable per family to avoid
# multicollinearity. The 30-day window is chosen as it balances
# noise (shorter windows) vs dilution of signal (longer windows).
#
# Features:
# - car_raw_30d: cumulative abnormal return, our key outcome
# - abnormal_volume_30d: unusual trading activity in the stock
# - realized_vol_30d: recent stock volatility
# - momentum_20d: price trend before trade
# - beta_252d: systematic risk exposure
# - amihud_illiq_20d: liquidity proxy (informed traders may prefer liquid stocks)
# - filing_delay: strategic disclosure timing
# ============================================================

features_model = [
    'car_raw_30d',        # Abnormal return (outcome of interest)
    'abnormal_volume_30d', # Volume anomaly at trade time
    'realized_vol_30d',    # Recent volatility
    'momentum_20d',        # Price momentum
    'beta_252d',           # Market sensitivity
    'amihud_illiq_20d',    # Illiquidity measure
    'filing_delay'         # Days to disclose trade
]

# Check availability
features_available = [f for f in features_model if f in df.columns]
features_missing = [f for f in features_model if f not in df.columns]

print(f'Features available: {len(features_available)}/{len(features_model)}')
if features_missing:
    print(f'Missing: {features_missing}')

In [ ]:
# Prepare modeling dataset
# Keep only rows with complete feature data

df_model = df[['trade_id', 'name', 'party_label', 'chamber_label', 'ticker', 
               'transaction', 'traded_date', 'trade_year', 'committee'] + features_available].copy()

print(f'Before dropping NAs: {len(df_model):,}')
df_model = df_model.dropna(subset=features_available)
print(f'After dropping NAs:  {len(df_model):,}')

# Handle infinities
X = df_model[features_available].copy()
X = X.replace([np.inf, -np.inf], np.nan)
valid_idx = X.dropna().index
df_model = df_model.loc[valid_idx]
X = X.loc[valid_idx]

print(f'Final sample: {len(df_model):,} trades')

In [ ]:
# ============================================================
# SCALING
# We use RobustScaler (median/IQR) instead of StandardScaler
# because financial data often has heavy tails and outliers.
# This prevents extreme values from dominating the scaling.
# ============================================================

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

print(f'Scaled feature matrix: {X_scaled.shape}')

---
## 4. Anomaly Detection Model

### Methodological Choices

**Why Isolation Forest + LOF?**

- **Isolation Forest** (Liu et al., 2008): Identifies points that are easy to isolate in random partitions. Works well for detecting global outliers—trades that are unusual compared to the entire dataset.

- **Local Outlier Factor** (Breunig et al., 2000): Compares local density around each point to its neighbors. Detects contextual anomalies—trades that are unusual relative to similar trades.

Using both methods provides robustness: trades flagged by both are more likely to be genuinely anomalous.

**Hyperparameters:**

- `contamination = 0.05`: We assume ~5% of trades may be anomalous. This is conservative; we conduct sensitivity analysis below.
- `n_estimators = 200`: Number of trees in IF. Literature suggests 100-300 is typically sufficient for convergence.
- `n_neighbors = 50`: For LOF, we use ~1-2% of the dataset, balancing local sensitivity vs stability.

In [ ]:
# ============================================================
# MODEL CONFIGURATION
# ============================================================

CONTAMINATION = 0.05  # Expected proportion of anomalies
RANDOM_STATE = 42     # For reproducibility

# IF: n_estimators=200 is standard; diminishing returns beyond ~300
N_ESTIMATORS = 200

# LOF: n_neighbors as ~1% of sample, minimum 20
N_NEIGHBORS = max(20, int(len(X_scaled) * 0.01))

print(f'Configuration:')
print(f'  Contamination: {CONTAMINATION}')
print(f'  IF n_estimators: {N_ESTIMATORS}')
print(f'  LOF n_neighbors: {N_NEIGHBORS}')

In [ ]:
# ============================================================
# ISOLATION FOREST
# ============================================================

clf_if = IForest(
    contamination=CONTAMINATION,
    n_estimators=N_ESTIMATORS,
    max_samples='auto',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

clf_if.fit(X_scaled)

df_model['score_if'] = clf_if.decision_scores_
df_model['anomaly_if'] = clf_if.labels_  # 1 = anomaly

n_anom_if = df_model['anomaly_if'].sum()
print(f'Isolation Forest: {n_anom_if:,} anomalies ({n_anom_if/len(df_model)*100:.1f}%)')

In [ ]:
# ============================================================
# LOCAL OUTLIER FACTOR
# ============================================================

clf_lof = LOF(
    n_neighbors=N_NEIGHBORS,
    contamination=CONTAMINATION,
    n_jobs=-1
)

clf_lof.fit(X_scaled)

df_model['score_lof'] = clf_lof.decision_scores_
df_model['anomaly_lof'] = clf_lof.labels_

n_anom_lof = df_model['anomaly_lof'].sum()
print(f'LOF: {n_anom_lof:,} anomalies ({n_anom_lof/len(df_model)*100:.1f}%)')

In [ ]:
# ============================================================
# COMBINED SCORE
# Normalize scores to [0,1] and average for a consensus measure.
# Flag as anomaly only if BOTH methods agree (conservative).
# ============================================================

scaler_mm = MinMaxScaler()
scores_norm = scaler_mm.fit_transform(df_model[['score_if', 'score_lof']])

df_model['score_if_norm'] = scores_norm[:, 0]
df_model['score_lof_norm'] = scores_norm[:, 1]
df_model['score_combined'] = (scores_norm[:, 0] + scores_norm[:, 1]) / 2

# Conservative flag: anomaly in BOTH methods
df_model['anomaly_both'] = ((df_model['anomaly_if'] == 1) & 
                            (df_model['anomaly_lof'] == 1)).astype(int)

n_anom_both = df_model['anomaly_both'].sum()
print(f'Consensus (both methods): {n_anom_both:,} anomalies ({n_anom_both/len(df_model)*100:.2f}%)')

In [ ]:
# Agreement between methods
agreement = (df_model['anomaly_if'] == df_model['anomaly_lof']).mean()
print(f'IF-LOF agreement rate: {agreement*100:.1f}%')

### 4.1 Anomaly Score Analysis

In [ ]:
# ============================================================
# FIGURE 5: Anomaly Score Distributions
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(13, 4))

# IF score
axes[0].hist(df_model['score_if_norm'], bins=50, color=COLORS['primary'], 
             edgecolor='white', alpha=0.8)
q95_if = df_model['score_if_norm'].quantile(0.95)
axes[0].axvline(q95_if, color=COLORS['accent'], linestyle='--', linewidth=2, label='95th pctl')
axes[0].set_xlabel('Anomaly Score (normalized)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Isolation Forest')
axes[0].legend()

# LOF score
axes[1].hist(df_model['score_lof_norm'], bins=50, color=COLORS['primary'], 
             edgecolor='white', alpha=0.8)
q95_lof = df_model['score_lof_norm'].quantile(0.95)
axes[1].axvline(q95_lof, color=COLORS['accent'], linestyle='--', linewidth=2, label='95th pctl')
axes[1].set_xlabel('Anomaly Score (normalized)')
axes[1].set_title('Local Outlier Factor')
axes[1].legend()

# Combined
axes[2].hist(df_model['score_combined'], bins=50, color=COLORS['primary'], 
             edgecolor='white', alpha=0.8)
q95_comb = df_model['score_combined'].quantile(0.95)
axes[2].axvline(q95_comb, color=COLORS['accent'], linestyle='--', linewidth=2, label='95th pctl')
axes[2].set_xlabel('Anomaly Score (normalized)')
axes[2].set_title('Combined Score')
axes[2].legend()

plt.suptitle('Distribution of Anomaly Scores', fontsize=13, y=1.02)
plt.tight_layout()
plt.savefig('fig5_score_distributions.png', dpi=300)
plt.show()

In [ ]:
# ============================================================
# FIGURE 6: IF vs LOF Score Comparison
# Shows whether both methods identify similar trades as anomalous
# ============================================================

fig, ax = plt.subplots(figsize=(7, 6))

# Plot normal points
normal = df_model[df_model['anomaly_both'] == 0]
ax.scatter(normal['score_if_norm'], normal['score_lof_norm'], 
           s=8, alpha=0.3, c=COLORS['secondary'], label='Normal')

# Plot anomalies
anomalies = df_model[df_model['anomaly_both'] == 1]
ax.scatter(anomalies['score_if_norm'], anomalies['score_lof_norm'], 
           s=25, alpha=0.8, c=COLORS['accent'], label='Anomaly (both)', edgecolors='white', linewidth=0.5)

# Reference lines at threshold
thresh = 1 - CONTAMINATION
ax.axvline(df_model['score_if_norm'].quantile(thresh), color=COLORS['primary'], 
           linestyle=':', alpha=0.5)
ax.axhline(df_model['score_lof_norm'].quantile(thresh), color=COLORS['primary'], 
           linestyle=':', alpha=0.5)

ax.set_xlabel('Isolation Forest Score')
ax.set_ylabel('LOF Score')
ax.set_title('Model Agreement: IF vs LOF Scores')
ax.legend(loc='lower right')

# Correlation annotation
r = df_model['score_if_norm'].corr(df_model['score_lof_norm'])
ax.text(0.05, 0.95, f'r = {r:.2f}', transform=ax.transAxes, fontsize=11,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.savefig('fig6_if_vs_lof.png', dpi=300)
plt.show()

---
## 5. Validation: Do Anomalies Exhibit Higher Abnormal Returns?

If our anomaly detection is capturing informed trading, we expect:
- Anomalous trades should have **higher CAR** on average
- The difference should be **statistically significant**

This serves as indirect validation since we lack ground-truth labels.

In [ ]:
# ============================================================
# TABLE 1: CAR Comparison - Normal vs Anomalous Trades
# ============================================================

def compare_groups(df, group_col, outcome_col):
    """Compare outcome between groups with t-test."""
    g0 = df[df[group_col] == 0][outcome_col].dropna()
    g1 = df[df[group_col] == 1][outcome_col].dropna()
    
    result = {
        'Normal (n)': len(g0),
        'Normal (mean)': g0.mean(),
        'Normal (median)': g0.median(),
        'Anomaly (n)': len(g1),
        'Anomaly (mean)': g1.mean(),
        'Anomaly (median)': g1.median(),
        'Diff (mean)': g1.mean() - g0.mean(),
        't-stat': stats.ttest_ind(g0, g1)[0],
        'p-value': stats.ttest_ind(g0, g1)[1]
    }
    return result

# Compare for different CAR windows
car_vars = ['car_raw_30d', 'car_capm_30d', 'car_raw_60d', 'car_capm_60d']
car_vars = [v for v in car_vars if v in df_model.columns]

results = []
for var in car_vars:
    res = compare_groups(df_model, 'anomaly_both', var)
    res['Outcome'] = var
    results.append(res)

results_df = pd.DataFrame(results)
results_df = results_df[['Outcome', 'Normal (n)', 'Anomaly (n)', 'Normal (mean)', 
                         'Anomaly (mean)', 'Diff (mean)', 't-stat', 'p-value']]

print('\nCAR Comparison: Normal vs Anomalous Trades')
print('='*80)
print(results_df.to_string(index=False))

In [ ]:
# ============================================================
# FIGURE 7: CAR Distribution by Anomaly Status
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(11, 4.5))

# Left: Boxplot
car_var = 'car_raw_30d' if 'car_raw_30d' in df_model.columns else car_vars[0]

box_data = [df_model[df_model['anomaly_both']==0][car_var].dropna(),
            df_model[df_model['anomaly_both']==1][car_var].dropna()]

bp = axes[0].boxplot(box_data, labels=['Normal', 'Anomaly'], patch_artist=True,
                     widths=0.6, showfliers=False)  # Hide outliers for clarity

bp['boxes'][0].set_facecolor(COLORS['secondary'])
bp['boxes'][1].set_facecolor(COLORS['accent'])
for box in bp['boxes']:
    box.set_alpha(0.7)

axes[0].axhline(0, color='black', linestyle='-', linewidth=0.5)
axes[0].set_ylabel('CAR 30d')
axes[0].set_title('Distribution of Abnormal Returns')

# Right: KDE comparison
normal_car = df_model[df_model['anomaly_both']==0][car_var].dropna()
anomaly_car = df_model[df_model['anomaly_both']==1][car_var].dropna()

# Clip for visualization
clip_low, clip_high = -0.3, 0.3
normal_clip = normal_car[(normal_car >= clip_low) & (normal_car <= clip_high)]
anomaly_clip = anomaly_car[(anomaly_car >= clip_low) & (anomaly_car <= clip_high)]

normal_clip.plot.kde(ax=axes[1], color=COLORS['secondary'], linewidth=2, label='Normal')
anomaly_clip.plot.kde(ax=axes[1], color=COLORS['accent'], linewidth=2, label='Anomaly')

axes[1].axvline(normal_car.mean(), color=COLORS['secondary'], linestyle='--', linewidth=1.5)
axes[1].axvline(anomaly_car.mean(), color=COLORS['accent'], linestyle='--', linewidth=1.5)
axes[1].axvline(0, color='black', linestyle='-', linewidth=0.5)

axes[1].set_xlabel('CAR 30d')
axes[1].set_ylabel('Density')
axes[1].set_title('Density Comparison')
axes[1].legend()
axes[1].set_xlim(clip_low, clip_high)

plt.suptitle('Cumulative Abnormal Returns: Normal vs Anomalous Trades', fontsize=13, y=1.02)
plt.tight_layout()
plt.savefig('fig7_car_comparison.png', dpi=300)
plt.show()

---
## 6. Sensitivity Analysis

We test robustness of results to different `contamination` rates.

In [ ]:
# ============================================================
# SENSITIVITY: Varying contamination rate
# Key question: Do anomalies consistently show higher CAR?
# ============================================================

contamination_values = [0.01, 0.02, 0.05, 0.10, 0.15]
sensitivity_results = []

car_col = 'car_raw_30d' if 'car_raw_30d' in df_model.columns else features_available[0]

for cont in contamination_values:
    # IF
    clf = IForest(contamination=cont, n_estimators=200, random_state=42)
    clf.fit(X_scaled)
    labels = clf.labels_
    
    n_anom = labels.sum()
    car_normal = df_model.loc[labels == 0, car_col].mean()
    car_anom = df_model.loc[labels == 1, car_col].mean()
    
    # T-test
    g0 = df_model.loc[labels == 0, car_col].dropna()
    g1 = df_model.loc[labels == 1, car_col].dropna()
    _, pval = stats.ttest_ind(g0, g1) if len(g1) > 1 else (np.nan, np.nan)
    
    sensitivity_results.append({
        'Contamination': cont,
        'N Anomalies': n_anom,
        '% Anomalies': n_anom / len(df_model) * 100,
        'CAR Normal': car_normal,
        'CAR Anomaly': car_anom,
        'Diff': car_anom - car_normal,
        'p-value': pval
    })

sens_df = pd.DataFrame(sensitivity_results)
print('Sensitivity Analysis: Isolation Forest')
print('='*70)
print(sens_df.round(4).to_string(index=False))

In [ ]:
# ============================================================
# FIGURE 8: Sensitivity Analysis Visualization
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(11, 4.5))

# Left: CAR difference by contamination
axes[0].bar(range(len(sens_df)), sens_df['Diff'], color=COLORS['primary'], edgecolor='white')
axes[0].set_xticks(range(len(sens_df)))
axes[0].set_xticklabels([f"{c:.0%}" for c in sens_df['Contamination']])
axes[0].axhline(0, color='black', linewidth=0.5)
axes[0].set_xlabel('Contamination Rate')
axes[0].set_ylabel('CAR Difference (Anomaly - Normal)')
axes[0].set_title('Effect Size Across Contamination Rates')

# Right: p-value by contamination
axes[1].plot(range(len(sens_df)), sens_df['p-value'], 'o-', color=COLORS['primary'], 
             markersize=8, linewidth=2)
axes[1].axhline(0.05, color=COLORS['accent'], linestyle='--', linewidth=1.5, label='α = 0.05')
axes[1].axhline(0.01, color=COLORS['warning'], linestyle='--', linewidth=1.5, label='α = 0.01')
axes[1].set_xticks(range(len(sens_df)))
axes[1].set_xticklabels([f"{c:.0%}" for c in sens_df['Contamination']])
axes[1].set_xlabel('Contamination Rate')
axes[1].set_ylabel('p-value')
axes[1].set_title('Statistical Significance')
axes[1].legend()
axes[1].set_ylim(0, max(0.1, sens_df['p-value'].max() * 1.1))

plt.suptitle('Sensitivity Analysis: Model Robustness', fontsize=13, y=1.02)
plt.tight_layout()
plt.savefig('fig8_sensitivity.png', dpi=300)
plt.show()

---
## 7. Characterization of Anomalous Trades

In [ ]:
# ============================================================
# TABLE 2: Feature Comparison - Normal vs Anomaly
# Which features differentiate anomalous trades?
# ============================================================

feature_comparison = []

for feat in features_available:
    normal_mean = df_model[df_model['anomaly_both']==0][feat].mean()
    anom_mean = df_model[df_model['anomaly_both']==1][feat].mean()
    
    # Percent difference
    if normal_mean != 0:
        pct_diff = (anom_mean - normal_mean) / abs(normal_mean) * 100
    else:
        pct_diff = np.nan
    
    # T-test
    g0 = df_model[df_model['anomaly_both']==0][feat].dropna()
    g1 = df_model[df_model['anomaly_both']==1][feat].dropna()
    _, pval = stats.ttest_ind(g0, g1) if len(g1) > 1 else (np.nan, np.nan)
    
    feature_comparison.append({
        'Feature': feat,
        'Normal Mean': normal_mean,
        'Anomaly Mean': anom_mean,
        'Diff %': pct_diff,
        'p-value': pval
    })

feat_df = pd.DataFrame(feature_comparison).sort_values('Diff %', key=abs, ascending=False)
print('Feature Comparison: Normal vs Anomalous Trades')
print('='*70)
print(feat_df.round(4).to_string(index=False))

In [ ]:
# ============================================================
# FIGURE 9: Feature Importance
# ============================================================

fig, ax = plt.subplots(figsize=(8, 5))

feat_df_sorted = feat_df.sort_values('Diff %')
colors = [COLORS['accent'] if x > 0 else COLORS['dem'] for x in feat_df_sorted['Diff %']]

ax.barh(feat_df_sorted['Feature'], feat_df_sorted['Diff %'], color=colors, edgecolor='white')
ax.axvline(0, color='black', linewidth=0.8)
ax.set_xlabel('Difference (%) Anomaly vs Normal')
ax.set_title('What Characterizes Anomalous Trades?')

# Mark significant features
for i, (_, row) in enumerate(feat_df_sorted.iterrows()):
    if row['p-value'] < 0.05:
        ax.annotate('*', xy=(row['Diff %'], i), fontsize=14, ha='left' if row['Diff %'] > 0 else 'right')

plt.tight_layout()
plt.savefig('fig9_feature_importance.png', dpi=300)
plt.show()

print('* indicates p < 0.05')

In [ ]:
# ============================================================
# Anomaly rate by party
# ============================================================

party_anomaly = df_model.groupby('party_label').agg({
    'anomaly_both': ['sum', 'mean', 'count']
}).round(4)
party_anomaly.columns = ['N Anomalies', 'Anomaly Rate', 'Total Trades']

print('Anomaly Rate by Party')
print('='*50)
print(party_anomaly)

In [ ]:
# ============================================================
# Top politicians by anomaly rate (min 20 trades)
# ============================================================

politician_stats = df_model.groupby('name').agg({
    'anomaly_both': ['sum', 'mean'],
    'score_combined': 'mean',
    'trade_id': 'count',
    'party_label': 'first'
})
politician_stats.columns = ['N_Anomalies', 'Anomaly_Rate', 'Avg_Score', 'Total_Trades', 'Party']
politician_stats = politician_stats[politician_stats['Total_Trades'] >= 20]

print('Top 15 Politicians by Anomaly Rate (min 20 trades)')
print('='*70)
print(politician_stats.nlargest(15, 'Anomaly_Rate').round(4))

In [ ]:
# ============================================================
# FIGURE 10: Top 15 Politicians by Anomaly Rate
# ============================================================

top15 = politician_stats.nlargest(15, 'Anomaly_Rate').reset_index()

fig, ax = plt.subplots(figsize=(10, 6))

colors = [COLORS['dem'] if p == 'Democrat' else COLORS['rep'] if p == 'Republican' else COLORS['ind'] 
          for p in top15['Party']]

bars = ax.barh(range(len(top15)), top15['Anomaly_Rate'] * 100, color=colors, edgecolor='white')
ax.set_yticks(range(len(top15)))
ax.set_yticklabels(top15['name'])
ax.set_xlabel('Anomaly Rate (%)')
ax.set_title('Politicians with Highest Anomaly Rates (min. 20 trades)')

# Add trade count annotation
for i, (_, row) in enumerate(top15.iterrows()):
    ax.annotate(f"n={row['Total_Trades']:.0f}", 
                xy=(row['Anomaly_Rate']*100 + 0.5, i),
                va='center', fontsize=9, color=COLORS['secondary'])

# Legend
legend_elements = [Line2D([0], [0], color=COLORS['dem'], lw=8, label='Democrat'),
                   Line2D([0], [0], color=COLORS['rep'], lw=8, label='Republican')]
ax.legend(handles=legend_elements, loc='lower right')

ax.invert_yaxis()
plt.tight_layout()
plt.savefig('fig10_top_politicians.png', dpi=300)
plt.show()

---
## 8. Export Results

In [ ]:
# Save dataset with anomaly scores
output_cols = ['trade_id', 'name', 'party_label', 'chamber_label', 'ticker', 
               'transaction', 'traded_date', 'trade_year', 'committee',
               'score_if', 'score_lof', 'score_combined',
               'anomaly_if', 'anomaly_lof', 'anomaly_both'] + features_available

output_cols = [c for c in output_cols if c in df_model.columns]

df_model[output_cols].to_csv('congress_anomaly_results.csv', index=False)
print('Saved: congress_anomaly_results.csv')

In [ ]:
# Summary statistics
print('\n' + '='*60)
print('SUMMARY')
print('='*60)
print(f'Sample size: {len(df_model):,} trades')
print(f'Features: {len(features_available)}')
print(f'Anomalies (IF): {df_model["anomaly_if"].sum():,} ({df_model["anomaly_if"].mean()*100:.1f}%)')
print(f'Anomalies (LOF): {df_model["anomaly_lof"].sum():,} ({df_model["anomaly_lof"].mean()*100:.1f}%)')
print(f'Anomalies (consensus): {df_model["anomaly_both"].sum():,} ({df_model["anomaly_both"].mean()*100:.2f}%)')
print(f'\nValidation:')
if car_col in df_model.columns:
    car_norm = df_model[df_model['anomaly_both']==0][car_col].mean()
    car_anom = df_model[df_model['anomaly_both']==1][car_col].mean()
    print(f'  CAR (normal):  {car_norm:.4f}')
    print(f'  CAR (anomaly): {car_anom:.4f}')
    print(f'  Difference:    {car_anom - car_norm:.4f}')
print('='*60)

---

## Notes for Next Steps

1. **Benchmark comparison (Ziobrowski)**: Build calendar-time portfolios and compute Fama-French alpha to compare aggregate abnormal returns.

2. **Supervised characterization**: Use Random Forest with `anomaly_both` as target to identify which features best predict anomalous trades.

3. **Committee analysis**: Test whether anomaly rates differ by committee membership, particularly for committees with access to market-moving information.

4. **Time-series patterns**: Check if anomaly rates increased around specific events (financial crises, policy announcements).